In [1]:
print("Hello World")

Hello World


## Import Libralies

In [10]:
import numpy as np
import random
import collections

## Utility Generation with Distributions

- This script provides a function to generate a set of cardinal utility values for agents and alternatives based on different probability distributions.

- Utility values represent the "true numerical value" an agent has for an alternative. These values are used to calculate the optimal social welfare, which is needed to
determine the distortion of a voting rule.

In [12]:
def generate_utilities(num_agents, num_alternatives, distribution_type='uniform'):
    """
    Generates a 2D array of cardinal utility values based on a specified distribution.

    Args:
        num_agents (int): The number of agents (voters).
        num_alternatives (int): The number of alternatives.
        distribution_type (str): The type of distribution to use.
                                 'uniform': Utilities are randomly and uniformly distributed between 0 and 1.
                                 'normal': Utilities follow a normal (Gaussian) distribution.
    Returns:
        list of list: A nested list where utilities[i][j] is the utility of alternative j for agent i.
    """
    utilities = []
    
    # Check for valid distribution type
    if distribution_type not in ['uniform', 'normal']:
        print(f"Warning: Invalid distribution type '{distribution_type}'. Defaulting to 'uniform'.")
        distribution_type = 'uniform'
        
    # Generate utilities based on the specified distribution
    for _ in range(num_agents):
        agent_utilities = []
        if distribution_type == 'uniform':
            # Uniform distribution: each utility value is equally likely to be between 0 and 1.
            agent_utilities = [random.uniform(0, 1) for _ in range(num_alternatives)]
        elif distribution_type == 'normal':
            # Normal distribution: values are clustered around a mean.
            # We use a mean of 0.5 and a standard deviation of 0.2 to keep values mostly between 0 and 1.
            agent_utilities = [np.random.normal(loc=0.5, scale=0.2) for _ in range(num_alternatives)]
            # We'll clip the values to ensure they stay within the [0, 1] range.
            agent_utilities = [max(0, min(1, u)) for u in agent_utilities]

        utilities.append(agent_utilities)
        
    return utilities

In [13]:
if __name__ == "__main__":
    # --- Example 1: Uniform Distribution ---
    num_agents_uniform = 10
    num_alternatives_uniform = 5
    uniform_utilities = generate_utilities(num_agents_uniform, num_alternatives_uniform, 'uniform')

    print("=== Uniform Distribution Utilities ===")
    print(f"Number of Agents: {num_agents_uniform}")
    print(f"Number of Alternatives: {num_alternatives_uniform}")
    print("Generated Utilities:")
    for i, u_list in enumerate(uniform_utilities):
        print(f"Agent {i+1}: {[round(u, 2) for u in u_list]}")

=== Uniform Distribution Utilities ===
Number of Agents: 10
Number of Alternatives: 5
Generated Utilities:
Agent 1: [0.36, 0.35, 0.72, 0.45, 0.6]
Agent 2: [0.82, 0.78, 0.23, 0.23, 0.01]
Agent 3: [0.22, 0.12, 0.86, 0.05, 0.11]
Agent 4: [0.71, 0.07, 0.66, 0.63, 0.24]
Agent 5: [0.38, 0.31, 0.62, 0.21, 0.84]
Agent 6: [0.67, 0.03, 0.79, 0.38, 0.06]
Agent 7: [0.31, 0.05, 0.12, 0.58, 0.65]
Agent 8: [0.36, 0.67, 0.21, 0.26, 0.92]
Agent 9: [0.98, 0.19, 0.07, 0.74, 0.29]
Agent 10: [0.71, 0.98, 0.53, 0.95, 0.37]


In [14]:
if __name__ == "__main__":    
# --- Example 2: Normal Distribution ---
    print("\n" + "="*40 + "\n")
    num_agents_normal = 10
    num_alternatives_normal = 5
    normal_utilities = generate_utilities(num_agents_normal, num_alternatives_normal, 'normal')

    print("=== Normal Distribution Utilities ===")
    print(f"Number of Agents: {num_agents_normal}")
    print(f"Number of Alternatives: {num_alternatives_normal}")
    print("Generated Utilities:")
    for i, u_list in enumerate(normal_utilities):
        print(f"Agent {i+1}: {[round(u, 2) for u in u_list]}")



=== Normal Distribution Utilities ===
Number of Agents: 10
Number of Alternatives: 5
Generated Utilities:
Agent 1: [0.43, 0, 0.41, 0.52, 0.55]
Agent 2: [0.29, 0.71, 0.52, 0.82, 0.32]
Agent 3: [0.39, 0.71, 0.61, 0.51, 0.47]
Agent 4: [0.55, 0.43, 0.62, 0.73, 0.38]
Agent 5: [0.65, 0.45, 0.49, 0.64, 0.43]
Agent 6: [0.44, 0.68, 0.75, 0.82, 0.53]
Agent 7: [0.67, 0, 0.36, 0.57, 0.41]
Agent 8: [0.42, 0.46, 0.36, 0.52, 0.4]
Agent 9: [0.61, 0.42, 0.44, 0.48, 0.48]
Agent 10: [0.75, 0.35, 0.46, 0.73, 0.58]


## Algorithm Implementation

## Plurality

- This script demonstrates the Plurality voting rule. It first generates a set of random
preferences for a given number of agents and alternatives. Then, it runs the Plurality
algorithm to find the winning alternative, which is the one with the most first-place votes.

In [16]:
def generate_random_preference_list(num_alternatives):
    """
    Generates a random permutation of alternatives to represent an agent's preferences.

    Args:
        num_alternatives (int): The total number of alternatives.

    Returns:
        list: A randomly ordered list of alternative names (e.g., ['A3', 'A1', 'A2']).
    """
    alternatives = [f'A{i + 1}' for i in range(num_alternatives)]
    random.shuffle(alternatives)
    return alternatives

def generate_preferences(num_agents, num_alternatives):
    """
    Simulates preferences for a given number of agents and alternatives.
    Each agent's preference is a random permutation of the alternatives.

    Args:
        num_agents (int): The number of agents (voters).
        num_alternatives (int): The number of alternatives.

    Returns:
        list: A list of lists, where each inner list is an agent's preference list.
    """
    preferences = []
    for _ in range(num_agents):
        preferences.append(generate_random_preference_list(num_alternatives))
    return preferences

def plurality_voting(preferences):
    """
    Implements the Plurality voting rule to find the winning alternative.
    Plurality works by giving one point to the first-ranked alternative of each agent.
    The alternative with the most points wins.

    Args:
        preferences (list): A 2D list of agent preferences.

    Returns:
        dict: A dictionary containing the winner, their vote count, and all scores.
    """
    if not preferences:
        return {"winner": None, "votes": 0, "scores": {}}

    # Tally the votes. Each agent's top-ranked choice gets one point.
    vote_counts = collections.Counter(pref[0] for pref in preferences)

    # Find the winner by identifying the alternative with the most votes
    winner = None
    max_votes = -1

    for alternative, votes in vote_counts.items():
        if votes > max_votes:
            max_votes = votes
            winner = alternative
        # Tie-breaking can be added here if needed, but for simplicity, the first found winner is returned.

    return {
        "winner": winner,
        "votes": max_votes,
        "scores": dict(vote_counts)
    }


In [17]:
# Run it
if __name__ == "__main__":
    num_agents = 10
    num_alternatives = 5

    # Generate preferences and run the Plurality algorithm
    agent_preferences = generate_preferences(num_agents, num_alternatives)
    results = plurality_voting(agent_preferences)

    # Print the results
    print("=== Plurality Voting Simulation ===")
    print(f"Number of Agents: {num_agents}")
    print(f"Number of Alternatives: {num_alternatives}")

    print("\nSimulated Preferences:")
    for i, pref in enumerate(agent_preferences):
        print(f"Agent {i + 1}: {' > '.join(pref)}")

    print("\nVoting Results:")
    print("Scores:", results['scores'])
    print(f"Winner: {results['winner']} with {results['votes']} votes.")

=== Plurality Voting Simulation ===
Number of Agents: 10
Number of Alternatives: 5

Simulated Preferences:
Agent 1: A5 > A2 > A3 > A1 > A4
Agent 2: A3 > A1 > A4 > A2 > A5
Agent 3: A1 > A4 > A5 > A2 > A3
Agent 4: A2 > A1 > A4 > A3 > A5
Agent 5: A4 > A1 > A3 > A2 > A5
Agent 6: A4 > A5 > A3 > A1 > A2
Agent 7: A3 > A2 > A5 > A4 > A1
Agent 8: A3 > A1 > A4 > A2 > A5
Agent 9: A2 > A4 > A5 > A3 > A1
Agent 10: A1 > A3 > A4 > A2 > A5

Voting Results:
Scores: {'A5': 1, 'A3': 3, 'A1': 2, 'A2': 2, 'A4': 2}
Winner: A3 with 3 votes.


# Veto

This script implements the Veto voting rule. It takes a set of agent preferences and
determines the winning alternative. 

The Veto rule works by giving one negative point
(a 'veto') to the last-ranked alternative of each agent. The alternative with the fewest
vetoes (the highest score) wins.

In [21]:
def veto_voting(preferences):
    """
    Implements the Veto voting rule to find the winning alternative.

    Args:
        preferences (list): A 2D list of agent preferences.

    Returns:
        dict: A dictionary containing the winner, their score, and all scores.
    """
    if not preferences or not preferences[0]:
        return {"winner": None, "score": 0, "scores": {}}

    num_alternatives = len(preferences[0])
    # The default score for all alternatives is zero.
    veto_scores = {alt: 0 for alt in preferences[0]}

    # Iterate through each agent's preferences.
    for preference_list in preferences:
        # The last-ranked alternative gets a negative point (or a veto).
        last_choice = preference_list[num_alternatives - 1]
        veto_scores[last_choice] += 1  # Increment the veto count

    # Find the winner: the alternative with the fewest vetoes.
    winner = None
    min_vetoes = float('inf')  # Start with a very high number to find the minimum.

    # Iterate through the scores to find the alternative with the minimum vetoes.
    for alternative, score in veto_scores.items():
        # The Veto rule is a bit counter-intuitive: the lowest score wins.
        if score < min_vetoes:
            min_vetoes = score
            winner = alternative
        # Tie-breaking can be added here if needed, but for simplicity, the first one is returned.
    
    # We can also represent the score positively for clarity (e.g., as 10 - number of vetoes)
    # This might make the output more readable.
    scores_for_display = {alt: len(preferences) - score for alt, score in veto_scores.items()}

    return {
        "winner": winner,
        "score": len(preferences) - min_vetoes,
        "scores": scores_for_display
    }

In [19]:
# Run it
if __name__ == "__main__":
    # You can reuse the generate_preferences function from your previous code.
    # For this example, let's use a sample preference list to demonstrate the logic.
    sample_preferences = [
        ['A1', 'A2', 'A3', 'A4'],  # A4 gets a veto
        ['A2', 'A3', 'A4', 'A1'],  # A1 gets a veto
        ['A4', 'A3', 'A2', 'A1'],  # A1 gets a veto
        ['A3', 'A1', 'A2', 'A4'],  # A4 gets a veto
        ['A2', 'A4', 'A1', 'A3'],  # A3 gets a veto
    ]
    
    print("=== Veto Voting Simulation ===")
    print("Simulated Preferences:")
    for i, pref in enumerate(sample_preferences):
        print(f"Agent {i + 1}: {' > '.join(pref)}")
    
    veto_results = veto_voting(sample_preferences)
    
    print("\nVeto Scores:")
    print(veto_results['scores'])
    print(f"Winner: {veto_results['winner']} with a score of {veto_results['score']}.")

=== Veto Voting Simulation ===
Simulated Preferences:
Agent 1: A1 > A2 > A3 > A4
Agent 2: A2 > A3 > A4 > A1
Agent 3: A4 > A3 > A2 > A1
Agent 4: A3 > A1 > A2 > A4
Agent 5: A2 > A4 > A1 > A3

Veto Scores:
{'A1': 3, 'A2': 5, 'A3': 4, 'A4': 3}
Winner: A2 with a score of 5.
